In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

In [39]:
# Load Data
uav = pd.read_csv('uav_flight_data.csv')

# TODO: Read the datetime from the parameters file
start = dt.datetime(year=2015,month=1,day=1)
uav['time'] = uav['time'] + start.timestamp()
uav["time"] = pd.to_datetime(uav["time"], unit='s')

uav = uav[['time', 'linear_acceleration_x', 'linear_acceleration_y', 'linear_acceleration_z', 'position_x']]

# Do I still need this for this clean dataset?
uav.fillna(uav.bfill())

# Extract the training and test datasets
train = uav[:int(len(uav)*0.8)]
test = uav[int(len(uav)*0.8):]

train_dates = pd.to_datetime(train['time'])
test_dates  = pd.to_datetime(test['time'])

print(train)
print(test)

                              time  linear_acceleration_x  \
0    2015-01-01 06:00:00.000000000               0.020421   
1    2015-01-01 06:00:00.100000000               0.021195   
2    2015-01-01 06:00:00.300000000               0.020522   
3    2015-01-01 06:00:00.410000128               0.019842   
4    2015-01-01 06:00:00.600000000               0.021527   
...                            ...                    ...   
1442 2015-01-01 06:03:36.200000000               0.266655   
1443 2015-01-01 06:03:36.400000000               0.292025   
1444 2015-01-01 06:03:36.500000000              -0.019332   
1445 2015-01-01 06:03:36.710000128               1.857258   
1446 2015-01-01 06:03:36.900000000              -0.299269   

      linear_acceleration_y  linear_acceleration_z  position_x  
0                 -0.119444              -9.806449  -79.782379  
1                 -0.104146              -9.805132  -79.782379  
2                 -0.119607              -9.806278  -79.782379  
3      

In [45]:
# Prepare Data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(train[['linear_acceleration_x', 'linear_acceleration_y', 'linear_acceleration_z', 'position_x']])

prediction_days = 225

X_train = []
y_train = []

for x in range(prediction_days, len(scaled_data)):
    x_train.append(scaled_data[x-prediction_days:x, 0])
    y_train.append(scaled_data[x, 0])


x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

[[0.48897588 0.43692092 0.52463933 0.28559377]
 [0.48915698 0.44051114 0.52482616 0.28558124]
 [0.48899932 0.43688277 0.52466363 0.28547282]
 ...
 [0.4796729  0.50786142 0.52361    0.29010891]
 [0.91882679 0.59740246 0.65213127 0.28911476]
 [0.4141629  0.59257283 0.54946038 0.28833141]]
1447


In [ ]:
# Build The Model
model = Sequential()

model.add(LSTM(units =128, activation='tanh', return_sequences=True, input_shape = (x_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(units =128, activation='tanh', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units =128, activation='tanh', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1)) # Prediction of the next value

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

## Need to figure out how the data size and the batch_size work ... need to optimize for a GPU system.  Higher batch size works much better on the GPU (less overhead)
history = model.fit(x_train, y_train, epochs = 25, batch_size=32, validation_split=0.1)


plt.plot(history.history['loss'], label = 'Training Loss')
plt.plot(history.history['val_loss'], label = 'Validation Loss')
plt.legend()
plt.show()

In [ ]:
### Test the Model Accuracy on Existing Data ###
################################################

# take Test data Again
test.head()
actual_temp = test['Avg_Temp'].values
total_temp = pd.concat((train['Avg_Temp'], test['Avg_Temp']),axis=0)

model_inputs = total_temp[len(total_temp)-len(test)-prediction_days:].values
model_inputs = model_inputs.reshape(-1,1)
model_inputs = scaler.transform(model_inputs)


# Make Predictions on Test Data
x_test = []

for x in range(prediction_days, len(model_inputs)):
    x_test.append(model_inputs[x-prediction_days:x, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

pred = model.predict(x_test)
print('Predictions:\n', pred[:][:10])
pred = scaler.inverse_transform(pred)
print('Inverse Transformed Predictions\n', pred[:][:10])

from sklearn.metrics import mean_absolute_error
mean_absolute_error(test['Avg_Temp'], pred)

In [ ]:
### Visualization ###
#####################
pred_ = pd.DataFrame(test['Date'])
pred_['Avg_Temp'] = pred
pred_["Date"] = pd.to_datetime(pred_["Date"])

pred_
original = florida.loc[florida['Date'] >= '1990-01-01']

import seaborn as sns
sns.lineplot(original['Date'], original['Avg_Temp'])
sns.lineplot(pred_['Date'], pred_['Avg_Temp'])
plt.show()